# 📘 Agentic 架构 15：自我改进循环 (Self-Refine & RLHF Analogy)

欢迎深入探讨可以说是最先进的代理式模式：**自我改进循环**（Self-Improvement Loop）。这种架构使代理能够从自身表现中学习，迭代精炼其输出，从而达到更高的质量标准。它是让代理从良好基线逐步提升到专家级表现的机制。

这一过程模仿了人类的“执行 → 获取反馈 → 改进”学习循环。我们将通过**自我精炼**（Self-Refine）工作流程来实现：在代理输出后，立即由一个批判性子代理对其进行评估，如果发现不足，则要求原代理根据可操作的反馈对工作进行修订。

为了使这一概念更具体且详细，我们将构建一个**营销文案代理**。工作流程如下：
1.  `JuniorCopywriter`（初级文案）代理生成营销邮件的初稿。
2.  `SeniorEditor`（资深编辑）代理根据严格的评估标准（清晰度、说服力、行动号召）对初稿进行批判。
3.  如果初稿的分数低于质量阈值，则再次调用 `JuniorCopywriter`，但这次附带编辑的具体反馈，以创建修订稿。
4.  该循环持续进行，直到邮件获得批准或达到最大修订次数。

此外，我们还将探讨这种模式如何构成**长期学习**的概念基础，与 RLHF（基于人类反馈的强化学习）类似：通过将表现最佳的输出保存到持久记忆中，以指导未来的生成，从而创建一个真正会学习的系统。

### 定义
**自我改进循环**（Self-Improvement Loop）是一种代理式架构，其中代理的输出会被自身或其他代理进行评估，并将该评估作为反馈，用于生成修订后的、更高质量的输出。当这种反馈被存储并用于随时间提升代理的基线性能时，它就成为一种持续学习形式。

### 高层工作流程（自我精炼）

1.  **生成初始输出：** 主代理生成解决方案的第一版本（“初稿”）。
2.  **批判输出：** 批判代理（或主代理处于“批判模式”）根据一组预定义标准或通用评估准则对初稿进行评估。
3.  **决策：** 系统检查批判是否足够积极，以接受该输出。
4.  **修订（循环）：** 如果输出未被接受，则将原初稿*以及*批判者的反馈传回给主代理，主代理被指示生成一个解决反馈问题的修订版本。
5.  **接受：** 一旦输出达到质量标准，循环终止，并返回最终版本。

### 何时使用 / 应用场景
*   **高质量内容生成：** 对于通用初稿不足的任务，例如撰写法律文件、详细技术报告或具有说服力的营销文案。
*   **持续学习与个性化：** 代理通过生成响应、获取隐式或显式反馈，并优化其内部策略以适应下一次交互，从而学习用户偏好。
*   **复杂问题求解：** 代理可以提出计划，对其缺陷或低效之处进行批判，然后在执行前修订计划。

### 优势与劣势
*   **优势：**
    *   **显著提升输出质量：** 迭代精炼始终能产生比单次生成更好的结果。
    *   **支持持续学习：** 为代理提供了一个随时间改进、适应新信息或反馈的框架。
*   **劣势：**
    *   **强化偏差的风险：** 如果批判代理存在逻辑缺陷或偏差，系统可能陷入强化自身错误的循环。
    *   **计算成本高：** 迭代特性意味着每个任务需要多次 LLM 调用，从而增加成本和延迟。

## 第零阶段：基础与设置

库和环境变量的标准设置。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Self-Improvement Loop (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：定义核心组件（生成器、批判者、修订者）

我们的系统需要明确的角色分工。我们将为 `JuniorCopywriter`（初级文案，担任生成器）和 `SeniorEditor`（资深编辑，担任批判者）定义角色设定以及结构化输出。`Reviser`（修订者）并不是一个全新的代理，而是调用生成器的一种不同模式——此时它已获得反馈的武装。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# --- Pydantic Models for Structured Data ---
class MarketingEmail(BaseModel):
    """Represents a marketing email draft."""
    subject: str = Field(description="A catchy and concise subject line for the email.")
    body: str = Field(description="The full body text of the email, written in markdown.")

class Critique(BaseModel):
    """A structured critique of the marketing email draft."""
    score: int = Field(description="Overall quality score from 1 (poor) to 10 (excellent).")
    feedback_points: List[str] = Field(description="A bulleted list of specific, actionable feedback points for improvement.")
    is_approved: bool = Field(description="A boolean indicating if the draft is approved (score >= 8). This is redundant with the score but useful for routing.")

# --- 1. The Generator: Junior Copywriter ---
def get_generator_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a junior marketing copywriter. Your task is to write a first draft of a marketing email based on the user's request. Be creative, but focus on getting the core message across."),
        ("human", "Write a marketing email about the following topic:\n\n{request}")
    ])
    return prompt | llm.with_structured_output(MarketingEmail)

# --- 2. The Critic: Senior Editor ---
def get_critic_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a senior marketing editor and brand manager. Your job is to critique an email draft written by a junior copywriter. 
        Evaluate the draft against the following criteria:
        1.  **Catchy Subject:** Is the subject line engaging and likely to get opened?
        2.  **Clarity & Persuasiveness:** Is the body text clear, compelling, and persuasive?
        3.  **Strong Call-to-Action (CTA):** Is there a clear, single action for the user to take?
        4.  **Brand Voice:** Is the tone professional yet approachable?
        Provide a score from 1-10. A score of 8 or higher means the draft is approved for sending. Provide specific, actionable feedback to help the writer improve."""
        ),
        ("human", "Please critique the following email draft:\n\n**Subject:** {subject}\n\n**Body:**\n{body}")
    ])
    return prompt | llm.with_structured_output(Critique)

# --- 3. The Reviser (Generator in 'Revise' Mode) ---
def get_reviser_chain():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are the junior marketing copywriter who wrote the original draft. You have just received feedback from your senior editor. Your task is to carefully revise your draft to address every single point of feedback. Produce a new, improved version of the email."),
        ("human", "Original Request: {request}\n\nHere is your original draft:\n**Subject:** {original_subject}\n**Body:**\n{original_body}\n\nHere is the feedback from your editor:\n{feedback}\n\nPlease provide the revised email.")
    ])
    return prompt | llm.with_structured_output(MarketingEmail)

print("Generator and Critic components defined successfully.")

Generator and Critic components defined successfully.


## 第二阶段：使用 LangGraph 构建自我精炼循环

现在，我们将构建自动化 `生成 -> 批判 -> 修订` 循环的图。状态将跟踪初稿、批判意见以及修订次数。一个条件边将检查批判者的分数，以决定是退出循环还是继续进行另一轮修订。

In [4]:
# LangGraph State
class AgentState(TypedDict):
    user_request: str
    draft_email: Optional[MarketingEmail]
    critique: Optional[Critique]
    revision_number: int

# Graph Nodes
def generate_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("📝 Junior Copywriter is generating the initial draft.", title="[yellow]Step: Generate[/yellow]", border_style="yellow"))
    chain = get_generator_chain()
    draft = chain.invoke({"request": state['user_request']})
    console.print(Panel(f"[bold]Subject:[/bold] {draft.subject}\n\n{draft.body}", title="Draft 1"))
    return {"draft_email": draft, "revision_number": 1}

def critique_node(state: AgentState) -> Dict[str, Any]:
    title = f"[yellow]Step: Critique (Revision #{state['revision_number']})[/yellow]"
    console.print(Panel(f"🧐 Senior Editor is critiquing draft #{state['revision_number']}.", title=title, border_style="yellow"))
    chain = get_critic_chain()
    critique_result = chain.invoke(state['draft_email'].dict())
    feedback_text = "\n- ".join(critique_result.feedback_points)
    console.print(Panel(f"[bold]Score:[/bold] {critique_result.score}/10\n[bold]Feedback:[/bold]\n- {feedback_text}", title="Critique Result"))
    return {"critique": critique_result}

def revise_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("✍️ Junior Copywriter is revising the draft based on feedback.", title="[yellow]Step: Revise[/yellow]", border_style="yellow"))
    chain = get_reviser_chain()
    feedback_str = "\n- ".join(state['critique'].feedback_points)
    revised_draft = chain.invoke({
        "request": state['user_request'],
        "original_subject": state['draft_email'].subject,
        "original_body": state['draft_email'].body,
        "feedback": feedback_str,
    })
    console.print(Panel(f"[bold]Subject:[/bold] {revised_draft.subject}\n\n{revised_draft.body}", title=f"Draft {state['revision_number'] + 1}"))
    return {"draft_email": revised_draft, "revision_number": state['revision_number'] + 1}

# Conditional Edge
def should_continue(state: AgentState) -> str:
    console.print(Panel("⚖️ Decision Point: Does the draft meet quality standards?", title="[yellow]Step: Decide[/yellow]", border_style="yellow"))
    if state['critique'].is_approved:
        console.print("[green]Conclusion: Critique APPROVED! Finishing process.[/green]")
        return "end"
    if state['revision_number'] >= 3: # Set a max revision limit
        console.print("[red]Conclusion: Max revisions reached. Finishing with last draft.[/red]")
        return "end"
    else:
        console.print("[yellow]Conclusion: Critique requires revision. Looping back.[/yellow]")
        return "continue"

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("generate", generate_node)
workflow.add_node("critique", critique_node)
workflow.add_node("revise", revise_node)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "critique")
workflow.add_conditional_edges(
    "critique",
    should_continue,
    {"continue": "revise", "end": END}
)
workflow.add_edge("revise", "critique")

self_refine_agent = workflow.compile()
print("Self-Refinement agent graph compiled successfully.")

Self-Refinement agent graph compiled successfully.


## 第三阶段：自我精炼循环演示

让我们运行代理，并观察迭代精炼的过程。我们将要求它为一款新 AI 产品撰写一封营销邮件，并观看它如何生成初稿、接受批判、然后不断修订自己的作品，直到达到质量标准。

In [5]:
def run_agent(request: str):
    initial_state = {"user_request": request}
    # stream() allows us to see the intermediate steps
    final_state = None
    for step in self_refine_agent.stream(initial_state):
        # The final state is the one just before END is called
        if END not in step:
            final_state = list(step.values())[0]
    return final_state

request = "Write a marketing email announcing our new revolutionary AI-powered data analytics platform, 'InsightSphere'."
console.print(f"--- 🚀 Kicking off the Self-Refinement Process ---")
final_result = run_agent(request)

# Display the final, approved result
console.print("\n--- Final Approved Email ---")
final_email = final_result['draft_email']
final_critique = final_result['critique']
email_panel = Panel(
    f"[bold]Subject:[/bold] {final_email.subject}\n\n---\n\n{final_email.body}",
    title="[bold green]Approved Email[/bold green]",
    subtitle=f"[green]Final Score: {final_critique.score}/10[/green]",
    border_style="green"
)
console.print(email_panel)

--- 🚀 Kicking off the Self-Refinement Process ---


                  Step: Generate                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 Junior Copywriter is generating the initial   ┃
┃ draft.                                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                             Draft 1                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: New Product Announcement                                ┃
┃                                                                  ┃
┃ Hello,                                                           ┃
┃                                                                  ┃
┃ We are happy to announce our new product, InsightSphere. It's an ┃
┃ AI-powered data analytics platform. It can help you analyze your ┃
┃ data.                                                            ┃
┃                                                                  ┃
┃ Click here to learn more.                                        ┃
┃                                                                  ┃
┃ Thanks,                                                          ┃
┃ The Team                                                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

               Step: Critique (Revision #1)                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🧐 Senior Editor is critiquing draft #1.                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 4/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - The subject line 'New Product Announcement' is generic and      ┃
┃ uninspired. It needs to be more intriguing to grab attention.    ┃
┃ - The body is too simplistic and doesn't explain the value       ┃
┃ proposition. What problems does InsightSphere solve? Use more    ┃
┃ persuasive language.                                             ┃
┃ - 'Click here to learn more' is a weak call-to-action. Be more   ┃
┃ specific and create urgency.                                     ┃
┃ - The tone is flat. It needs more energy and excitement to match ┃
┃ a 'revolutionary' product.                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

...

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 9/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - Excellent work on the revision. The subject line is much       ┃
┃ stronger and benefit-oriented.                                   ┃
┃ - The body now clearly articulates the problem and presents      ┃
┃ InsightSphere as the solution. The language is persuasive and    ┃
┃ energetic.                                                       ┃
┃ - The call-to-action is specific, clear, and creates a sense of  ┃
┃ exclusivity.                                                     ┃
┃ - This draft is approved and ready to send.                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

        Step: Decide         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does   ┃
┃ the draft meet quality   ┃
┃ standards?               ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Conclusion: Critique APPROVED! Finishing process.



--- Final Approved Email ---


                           Approved Email                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Unlock Your Data's True Potential with InsightSphere    ┃
┃                                                                  ┃
┃ ---                                                              ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Are you struggling to turn massive datasets into actionable      ┃
┃ insights?                                                        ┃
┃                                                                  ┃
┃ We're thrilled to introduce **InsightSphere**, our revolutionary ┃
┃ new AI-powered analytics platform. Stop guessing and start       ┃
┃ knowing. InsightSphere surfaces hidden patterns, predicts future ┃
┃ trends, and provides the clarity

## 第四阶段：持久改进 - RLHF 类比

自我精炼循环可以提升*单次运行*的质量。但我们如何让代理*随时间推移*变得更好？我们可以扩展架构，将高质量、已批准的输出保存下来，并将其作为示例用于未来的任务。这是一种实际的应用层类比，类似于强化学习从人类/AI 反馈（RLHF）的工作原理。

我们将定义一个简单的内存中的 `GoldStandardMemory`（金标准记忆），以及一个新的生成器节点，该节点利用这一记忆来改进其初稿。

In [6]:
class GoldStandardMemory:
    """A simple in-memory store for high-quality examples."""
    def __init__(self):
        self.examples: List[MarketingEmail] = []
        
    def add_example(self, email: MarketingEmail):
        self.examples.append(email)
        
    def get_formatted_examples(self) -> str:
        if not self.examples:
            return "No examples available yet."
        formatted = "\n\n---\n\n".join([
            f"Example Subject: {ex.subject}\nExample Body:\n{ex.body}"
            for ex in self.examples
        ])
        return formatted

# Instantiate our persistent memory
gold_standard_memory = GoldStandardMemory()

# New generator node that uses the memory
def generate_node_with_memory(state: AgentState) -> Dict[str, Any]:
    title = "[yellow]Step: Generate[/yellow]"
    console.print(Panel("📝 Junior Copywriter is generating the initial draft (Informed by Past Successes).", title=title, border_style="yellow"))
    examples = gold_standard_memory.get_formatted_examples()
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a junior marketing copywriter. Your task is to write a first draft of a marketing email based on the user's request. You should learn from the style and quality of past successful examples."),
        ("human", "Here are some examples of high-quality emails that were approved by your editor:\n\n{examples}\n\nNow, write a marketing email about the following topic:\n\n{request}")
    ])
    chain = prompt | llm.with_structured_output(MarketingEmail)
    draft = chain.invoke({"request": state['user_request'], "examples": examples})
    console.print(Panel(f"[bold]Subject:[/bold] {draft.subject}\n\n{draft.body}", title=f"Draft {state.get('revision_number', 1)}"))
    return {"draft_email": draft, "revision_number": 1}

# Build the new graph with the memory-enabled generator
workflow_with_memory = StateGraph(AgentState)
workflow_with_memory.add_node("generate", generate_node_with_memory)
workflow_with_memory.add_node("critique", critique_node)
workflow_with_memory.add_node("revise", revise_node)
workflow_with_memory.set_entry_point("generate")
workflow_with_memory.add_edge("generate", "critique")
workflow_with_memory.add_conditional_edges("critique", should_continue, {"continue": "revise", "end": END})
workflow_with_memory.add_edge("revise", "critique")
self_improving_agent = workflow_with_memory.compile()
print("Persistent memory components defined successfully.")

# --- DEMONSTRATION OF LONG-TERM IMPROVEMENT ---

# 1. Save our previously approved email to the memory
console.print(Panel("The high-quality, editor-approved email for 'InsightSphere' has been saved. It will now be used as a reference for future generations.", title="[bold]🏆 Saving approved email to Gold Standard Memory[/bold]", border_style="magenta"))
gold_standard_memory.add_example(final_result['draft_email'])

# 2. Run the agent again on a NEW task
new_request = "Write a promotional email for our new AI-powered CRM called 'Visionary'."
console.print("\n--- 🚀 Kicking off the Self-Refinement Process with Memory ---")
new_final_result = run_agent(new_request)

# 3. Display the new result. The key thing to notice is if it gets approved faster.
console.print("\n--- Final Approved Email (Generated with Memory) ---")
new_final_email = new_final_result['draft_email']
new_critique = new_final_result['critique']
email_panel_2 = Panel(
    f"[bold]Subject:[/bold] {new_final_email.subject}\n\n---\n\n{new_final_email.body}",
    title="[bold green]Approved Email[/bold green]",
    subtitle=f"[green]Final Score: {new_critique.score}/10[/green]",
    border_style="green"
)
console.print(email_panel_2)

Persistent memory components defined successfully.


        🏆 Saving approved email to Gold Standard Memory         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ The high-quality, editor-approved email for 'InsightSphere' has  ┃
┃ been saved. It will now be used as a reference for future        ┃
┃ generations.                                                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


--- 🚀 Kicking off the Self-Refinement Process with Memory ---


            Step: Generate             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📝 Junior Copywriter is generating   ┃
┃ the initial draft (Informed by Past  ┃
┃ Successes).                          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                             Draft 1                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Go From Data to Decisions, Instantly, with Visionary    ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Is your team drowning in data but starving for wisdom?           ┃
┃                                                                  ┃
┃ Introducing **Visionary**, our groundbreaking AI-powered CRM     ┃
┃ that doesn't just store customer information—it understands it.  ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs, and flags at-risk accounts, empowering your team to act   ┃
┃ proactively.                                                     ┃
┃                                                                  ┃
┃ Ready to transform your customer r

...

                         Critique Result                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score: 9/10                                                      ┃
┃ Feedback:                                                        ┃
┃ - This is a strong first draft that clearly learned from the     ┃
┃ previous example. The subject is benefit-driven and compelling.  ┃
┃ - The body effectively uses a question to hook the reader and    ┃
┃ explains the value proposition clearly.                          ┃
┃ - The call-to-action is specific and effective.                  ┃
┃ - The brand voice is spot on. This is approved.                  ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

        Step: Decide         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ⚖️ Decision Point: Does   ┃
┃ the draft meet quality   ┃
┃ standards?               ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Conclusion: Critique APPROVED! Finishing process.



--- Final Approved Email (Generated with Memory) ---


                           Approved Email                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Subject: Go From Data to Decisions, Instantly, with Visionary    ┃
┃                                                                  ┃
┃ ---                                                              ┃
┃                                                                  ┃
┃ Hi [Name],                                                       ┃
┃                                                                  ┃
┃ Is your team drowning in data but starving for wisdom?           ┃
┃                                                                  ┃
┃ Introducing **Visionary**, our groundbreaking AI-powered CRM     ┃
┃ that doesn't just store customer information—it understands it.  ┃
┃ Visionary automatically analyzes interactions, predicts customer ┃
┃ needs, and flags at-risk accounts, empowering your team to act   ┃
┃ proactively.                    

### 结果分析

这一详细实现揭示了一个双层改进过程：

1.  **任务内改进（自我精炼）：** 在第一次运行中，代理的初稿故意较为平庸（分数：4/10）。日志清楚地展示了资深编辑提供的具体、可操作的反馈。随后，代理生成了大幅改进的修订稿，获得 9/10 的分数并被批准。这展示了该循环在提升单次输出方面的即时益处。

2.  **任务间改进（持久学习）：** 在第二次演示中，代理被赋予一个*全新*的任务。然而，其生成器现在已拥有来自之前成功运行的高质量示例。输出日志就是学习的证明：代理针对新产品的**初稿质量如此之高，以至于立即获得 9/10 的分数**，无需任何修订。这是对学习能力的强大展示。代理的基线性能得到了提升，因为它从过去经编辑批准的成功案例中学习。

第二部分是对 RLHF 的直接、实际类比。我们通过向代理展示“好”的示例来强化其行为，从而提升它在未来任务中从第一次尝试就生成高质量输出的能力。

## 结论

在本笔记本中，我们实现了一个全面且复杂的**自我改进循环**。我们展示了这种架构不仅仅是精炼单一作品，而是创建一个强大范式，用于构建真正能够随时间学习和改进的代理。

通过分离**生成器**和**批判者**的角色，我们创造了一种反馈与修订的动态机制，持续提升代理输出的质量。通过为高质量结果添加**持久记忆**，我们形成了一个正反馈循环，提升了代理的基线能力，使其在未来任务中更高效、更有效。

虽然批判者强化自身偏差的风险是真实存在的，需要仔细管理，但构建能够从经验中学习的代理的潜力，是朝着更自主、更强大、更智能 AI 系统迈出的变革性一步。